In [1]:
import numpy as np 
import matplotlib.pyplot as plt

# install
## numpy
## matplotlib

### data load & preprocessing

In [2]:
from dataset.mnist import load_mnist

(train_raw_img, train_label), (test_raw_img, test_label) = load_mnist(flatten=False, normalize=False)
print(train_raw_img.shape)
print(train_label.shape)

(60000, 1, 28, 28)
(60000,)


In [3]:
np.unique(train_label, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]))

In [4]:
# train_dataset split according to the number

new_train_img = [[] for _ in range(10)]
new_train_label = [[] for _ in range(10)]

for i in range(len(train_label)) :
    new_train_img[train_label[i]].append(train_raw_img[i])
    new_train_label[train_label[i]].append(train_label[i])

# print(len(new_train_img[0])) # 0에 해당하는 image 개수
# print(new_train_img[0][0].shape) # 0에 해당하는 image중 첫번째 image의 shape

### 1. Create a classifier that distinguishes between zero and non-zero (using logistic regression)

In [5]:
def make_sample(idx) :
    sample_img = []
    sample_label = []
    
    # data sampling 
    for i in range(10) :
        if i == idx :
            sample_img += new_train_img[i][:1000]
            sample_label += (new_train_label[i][:1000])
        else :
            sample_img += new_train_img[i][:111]
            sample_label += (new_train_label[i][:111])

    sample_img = np.array(sample_img)
    sample_label = np.array(sample_label)
    
    # normalization (set value 0 ~ 1)
    sample_img = sample_img.astype('float')/255
    
    # target number는 1, 아니면 0
    sample_label = np.where(sample_label==idx, 1 ,0)
    
    # reshape
    sample_img = sample_img.reshape(len(sample_img.squeeze()), -1)
    sample_label = sample_label.reshape(len(sample_label.squeeze()), -1)
    
    return sample_img, sample_label

In [6]:
train_X, train_y = make_sample(idx = 0) # idx = target number
train_X = np.insert(train_X, 0, 1, axis=1) # bias 추가

# print(train_y.shape)

In [7]:
# cross entropy loss
def CrossEntropyLoss(w, X, y) :
    delta = 1e-7
    
    preds = 1 / (1+np.exp(-X.dot(w)))
    loss = np.sum(-y*np.log(preds+delta)-(1-y)*np.log(1-preds+delta))/len(preds)
        
    return loss

In [8]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [30]:
# train

def train(X, y) :
    """_summary_

    Args:
        X : train_X
        y : train_y

    Returns:
        w : weight
    """
    w = np.random.randn(len(X[0]), 1)
    lr = 0.1 # learning rate(수정 가능)
    step = 0
    acc = 0
    
    while (acc <= 0.8) :
        step += 1
        correct = 0
        
        y_pred = sigmoid(X@w)
        grad = X.T @ (y_pred - y) / X.shape
        w -= lr * grad 
        
        loss = CrossEntropyLoss(w, X, y)
        
        if loss > 0.5:
            correct += 1
            
        elif abs(loss - prev_loss) < 1e-4:
            break
            
        acc = correct/len(X)
        
        prev_loss = loss
        step += 1
        
        
        print("total step : %d " % step)
        print("error : %f, accuarcy : %f" % (loss, acc))
        
    return w

In [31]:
# save weight

w = train(train_X, train_y)

6.080767343078435


ValueError: non-broadcastable output operand with shape (785,1) doesn't match the broadcast shape (785,2)

In [10]:
# eval (accuracy)

def eval(idx, w) :
    """_summary_

    Args:
        idx : target_number
        w : parameter
    """
    test_X = test_raw_img.astype('float')/255    
    test_X = test_X.reshape(len(test_X.squeeze()), -1)
    test_X = np.insert(test_X, 0, 1, axis=1) # + bias

    test_y = np.where(test_label==idx, 1 ,0)
    test_y = test_y.reshape(len(test_y.squeeze()), -1)
    
    preds = 1/(1+np.exp(-test_X.dot(w)))
    result = np.where(preds>0.5, 1, 0)
    
    acc = np.sum(np.where(result==test_y, True, False))/len(preds)
    print('accuracy : ', acc)

eval(idx=0, w=test_w)

accuracy :  0.902


### 2. multi class single label classification (using logistic regression)

In [ ]:
# train several binary linear regression classifier (0~9)

In [ ]:
# eval

test_X = test_raw_img.astype('float')/255    
test_X = test_X.reshape(len(test_X.squeeze()), -1)
test_X = np.insert(test_X, 0, 1, axis=1) # + bias
print('shape of test_X : ', test_X.shape)
print('shape of test_label : ', test_label.shape)

# make prediction using argmax
max_pred = None # shape : (10000,1) or (10000,)

acc = np.sum(np.where(test_label==max_pred, True, False))/len(test_X)
print('accuracy : ', acc)

shape of test_X :  (10000, 785)
shape of test_label :  (10000,)
accuracy :  0.0


### L2 Regularization